# Tuto **Scrapy**

https://scrapy.org/

https://docs.scrapy.org/en/latest/

Scrapy is a fast high-level [**web crawling**](https://en.wikipedia.org/wiki/Web_crawler) (*indexation*) and [**web scraping**](https://en.wikipedia.org/wiki/Web_scraping) (*moissonage*) framework, used to crawl websites and extract structured data from their pages. It can be used for a wide range of purposes, from data mining to monitoring and automated testing.

# Premiers pas

## [**Coup d'oeil rapide sur Scrapy**](https://docs.scrapy.org/en/latest/intro/overview.html)

Scrapy (/ˈskreɪpaɪ/) est un cadre d'application pour explorer des sites Web et extraire des données structurées qui peuvent être utilisées pour un large éventail d'applications utiles, comme l'exploration de données, le traitement de l'information ou l'archivage historique.

Même si Scrapy a été conçu à l'origine pour le scraping Web, il peut également être utilisé pour extraire des données à l'aide d'API (telles que Amazon Associates Web Services) ou en tant que robot d'exploration Web à usage général.

### Présentation d'un exemple d'araignée

Afin de vous montrer ce que Scrapy apporte à la table, nous vous présenterons un exemple de Scrapy Spider utilisant le moyen le plus simple de faire fonctionner une araignée.

Voici le code d'une araignée qui extrait des citations célèbres du site Web https://quotes.toscrape.com, en suivant la pagination :

In [2]:
import scrapy

class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    start_urls = [
        'https://quotes.toscrape.com/tag/humor/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'author': quote.xpath('span/small/text()').get(),
                'text': quote.css('span.text::text').get(),
            }

        next_page = response.css('li.next a::attr("href")').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

Mettez ceci dans un fichier texte, nommez-le par exemple `quotes_spider.py` et exécutez l'araignée en utilisant la commande `runspider` :

    scrapy runspider quotes_spider.py -o quotes.jsonl

Lorsque cela se termine, vous aurez dans le fichier `quotes.jsonl` une liste des citations au format JSON Lines, contenant le texte et l'auteur, ressemblant à ceci :

    {"author": "Jane Austen", "text": "\u201cThe person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.\u201d"}
    {"author": "Steve Martin", "text": "\u201cA day without sunshine is like, you know, night.\u201d"}
    {"author": "Garrison Keillor", "text": "\u201cAnyone who thinks sitting in church can make you a Christian must also think that sitting in a garage can make you a car.\u201d"}
    ...

### Qu'est-ce qui vient de se passer?

Lorsque vous avez exécuté la commande `scrapy runspider quotes_spider.py`, Scrapy a recherché une définition Spider à l'intérieur et l'a exécutée via son moteur d'indexation.

L'exploration a commencé par envoyer des requêtes aux URL définies dans l'attribut `start_urls` (dans ce cas, uniquement l'URL pour les citations dans la catégorie humour) et a appelé la méthode de rappel par défaut `parse`, en passant l'objet de réponse comme argument. Dans le rappel de `parse`, nous parcourons les éléments de citation à l'aide d'un sélecteur CSS, produisons un dict Python avec le texte de citation et l'auteur extraits, recherchons un lien vers la page suivante et planifions une autre demande en utilisant la même méthode `parse` pour le rappel.

Ici, vous remarquez l'un des principaux avantages de Scrapy : les requêtes sont [planifiées et traitées de manière asynchrone](https://docs.scrapy.org/en/latest/topics/architecture.html#topics-architecture). Cela signifie que Scrapy n'a pas besoin d'attendre qu'une demande soit terminée et traitée, il peut envoyer une autre demande ou faire d'autres choses entre-temps. Cela signifie également que d'autres requêtes peuvent continuer même si certaines requêtes échouent ou si une erreur se produit lors de leur traitement.

Bien que cela vous permette d'effectuer des analyses très rapides (en envoyant plusieurs requêtes simultanées en même temps, de manière tolérante aux pannes), Scrapy vous permet également de contrôler la politesse de l'analyse via [quelques paramètres](https://docs.scrapy.org/en/latest/topics/settings.html#topics-settings-ref). Vous pouvez faire des choses comme définir un délai de téléchargement entre chaque requête, limiter le nombre de requêtes simultanées par domaine ou par adresse IP, et même utiliser une [extension de limitation automatique](https://docs.scrapy.org/en/latest/topics/autothrottle.html#topics-autothrottle) qui essaie de les comprendre automatiquement.

📌 Ceci utilise les [exportations de flux](https://docs.scrapy.org/en/latest/topics/feed-exports.html#topics-feed-exports) pour générer le fichier JSON, vous pouvez facilement changer le format d'exportation (XML ou CSV, par exemple) ou le backend de stockage (FTP ou Amazon S3, par exemple). Vous pouvez également écrire un [pipeline d'éléments](https://docs.scrapy.org/en/latest/topics/item-pipeline.html#topics-item-pipeline) pour stocker les éléments dans une base de données.

### Quoi d'autre?

Vous avez vu comment extraire et stocker des éléments d'un site Web à l'aide de Scrapy, mais ce n'est que la surface. Scrapy fournit de nombreuses fonctionnalités puissantes pour rendre le moissonage facile et efficace, telles que :

* Prise en charge intégrée de la [sélection et de l'extraction](https://docs.scrapy.org/en/latest/topics/selectors.html#topics-selectors) de données à partir de sources HTML/XML à l'aide de sélecteurs CSS étendus et d'expressions XPath, avec des méthodes d'assistance pour extraire à l'aide d'expressions régulières.
* Une [console shell interactive](https://docs.scrapy.org/en/latest/topics/shell.html#topics-shell) (compatible IPython) pour essayer les expressions CSS et XPath pour récupérer des données, très utile lors de l'écriture ou du débogage de vos spiders.
* Une prise en charge intégrée pour la [génération des exportations de flux](https://docs.scrapy.org/en/latest/topics/feed-exports.html#topics-feed-exports) dans plusieurs formats (JSON, CSV, XML) et les stocker dans plusieurs backends (FTP, S3, système de fichiers local)
* Prise en charge robuste de l'encodage et détection automatique, pour traiter les déclarations d'encodage étrangères, non standard et cassées.
* [Prise en charge d'une extensibilité solide](https://docs.scrapy.org/en/latest/index.html#extending-scrapy), vous permettant de brancher vos propres fonctionnalités à l'aide de [signaux](https://docs.scrapy.org/en/latest/topics/signals.html#topics-signals) et d'une API bien définie (middlewares, [extensions](https://docs.scrapy.org/en/latest/topics/extensions.html#topics-extensions) et [pipelines](https://docs.scrapy.org/en/latest/topics/item-pipeline.html#topics-item-pipeline)).
* Large gamme d'extensions intégrées et de middlewares pour gérer :
    * cookies et gestion des sessions
    * Fonctionnalités HTTP comme la compression, l'authentification, la mise en cache
    * usurpation d'agent utilisateur
    * robots.txt
    * restriction de profondeur d'exploration
    * et plus
* Une [console Telnet](https://docs.scrapy.org/en/latest/topics/telnetconsole.html#topics-telnetconsole) pour se connecter à une console Python exécutée dans votre processus Scrapy, pour introspecter et déboguer votre crawler
* De plus, d'autres avantages tels que des araignées réutilisables pour explorer des sites à partir de [plans de site](https://www.sitemaps.org/index.html) et de flux XML/CSV, un pipeline multimédia pour [télécharger automatiquement des images](https://docs.scrapy.org/en/latest/topics/media-pipeline.html#topics-media-pipeline) (ou tout autre média) associées aux éléments supprimés, un résolveur DNS de mise en cache, et bien plus encore !

#### Et après?

Les prochaines étapes pour vous consistent à installer Scrapy, à suivre le didacticiel pour apprendre à créer un projet Scrapy complet et à rejoindre la communauté. Merci de votre intérêt!

## [**Installation**](https://docs.scrapy.org/en/latest/intro/install.html)

## [**Didacticiel**](https://docs.scrapy.org/en/latest/intro/tutorial.html)

Dans ce didacticiel, nous supposerons que Scrapy est déjà installé sur votre système. Si ce n'est pas le cas, consultez le guide d'installation.

Nous allons gratter [quotes.toscrape.com](https://quotes.toscrape.com/), un site qui répertorie les citations d'auteurs célèbres.

Ce didacticiel vous guidera à travers ces tâches :
1. Créer un nouveau projet Scrapy
2. Écrire une araignée pour explorer un site et extraire des données
3. Exporter les données récupérées à l'aide de la ligne de commande
4. Changer l'araignée pour suivre récursivement les liens
5. Utiliser des arguments d'araignée

Scrapy est écrit en Python. Si vous débutez dans le langage, vous voudrez peut-être commencer par vous faire une idée de ce à quoi ressemble le langage, pour tirer le meilleur parti de Scrapy.

Si vous connaissez déjà d'autres langages et que vous souhaitez apprendre Python rapidement, le didacticiel Python est une bonne ressource.

Si vous débutez en programmation et que vous souhaitez commencer avec Python, les livres suivants peuvent vous être utiles :
* Automatisez les trucs ennuyeux avec Python
* Comment penser comme un informaticien
* Apprendre Python 3 à la dure

Vous pouvez également consulter cette liste de ressources Python pour les non-programmeurs, ainsi que les ressources suggérées dans le learnpython-subreddit.

### Créer un projet

Avant de commencer à gratter, vous devrez configurer un nouveau projet Scrapy. Entrez un répertoire dans lequel vous souhaitez stocker votre code et exécutez :

    scrapy startproject tutorial

Cela créera un répertoire `tutorial` avec le contenu suivant :

    tutorial/
        scrapy.cfg            # deploy configuration file
        tutorial/             # project's Python module, you'll import your code from here
            __init__.py
            items.py          # project items definition file
            middlewares.py    # project middlewares file
            pipelines.py      # project pipelines file
            settings.py       # project settings file
            spiders/          # a directory where you'll later put your spiders
                __init__.py

### Notre première araignée

Les araignées sont des classes que vous définissez et que Scrapy utilise pour extraire des informations d'un site Web (ou d'un groupe de sites Web). Ils doivent sous-classer `Spider` et définir les requêtes initiales à effectuer, éventuellement comment suivre les liens dans les pages et comment analyser le contenu de la page téléchargée pour extraire des données.

C'est le code de notre premier Spider. Enregistrez-le dans un fichier nommé `quotes_spider.py` sous le répertoire `tutorial/spiders` de votre projet :